<a href="https://colab.research.google.com/github/bonareri/Bitcoin-Prediction-Analysis/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install python-dotenv

In [8]:
!pip install tweepy

In [28]:
# import libraries
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from google.colab import files
plt.style.use('fivethirtyeight')

In [29]:
uploaded = files.upload()

Saving twitter_keys.env to twitter_keys (1).env


In [30]:
import os

print(os.listdir())  # Should display 'twitter_keys.env'

['.config', 'tweets.csv', 'twitter_keys.env', 'twitter_keys (1).env', '.ipynb_checkpoints', 'sample_data']


In [31]:
from dotenv import load_dotenv

# Load API keys from .env file
load_dotenv("twitter_keys.env")

# Access keys
BEARER_TOKEN = os.getenv("BEARER_TOKEN")

In [32]:
# Authenticate using Bearer Token (API v2)
client = tweepy.Client(bearer_token=BEARER_TOKEN)

In [33]:
query = "Bitcoin -is:retweet lang:en"  # Remove retweets, only English tweets
tweets = client.search_recent_tweets(query=query, max_results=10, tweet_fields=["created_at"])

# Store tweets in a list
tweet_data = [{"text": tweet.text, "created_at": tweet.created_at} for tweet in tweets.data]

# Print stored tweets
print(tweet_data)

TooManyRequests: 429 Too Many Requests
Too Many Requests

In [26]:
import csv

# Save tweets to a CSV file
with open("tweets.csv", "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Created_At", "Tweet_Text"])  # Header row
    for tweet in tweet_data:
        writer.writerow([tweet["created_at"], tweet["text"]])

print("Tweets saved successfully to tweets.csv")

NameError: name 'tweet_data' is not defined

### Preprocess the Tweets

In [25]:
import re

def clean_tweet(text):
    text = re.sub(r'@\w+', '', text)  # Remove mentions (@user)
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)  # Remove special characters
    text = text.lower().strip()  # Convert to lowercase and remove leading/trailing spaces
    return text

# Example usage
tweets = [clean_tweet(tweet) for tweet in raw_tweets]  # raw_tweets is your list of tweets

NameError: name 'raw_tweets' is not defined